In [9]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet


def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info


if __name__ == '__main__':
    for name, val in collect_env().items():
        print(f'{name}: {val}')

sys.platform: linux
Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
CUDA available: True
numpy_random_seed: 42
GPU 0: Tesla V100-SXM2-32GB
CUDA_HOME: None
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
PyTorch: 1.12.1+cu116
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86
  - CuDNN 8.3.2  (built against CUDA 11.5)
  

In [11]:
from pycocotools.coco import COCO

# Path to load the COCO annotation file
annotation_file = '../dataset/json/train.json'

# Initialise the COCO object
coco = COCO(annotation_file)

# Get all category tags and corresponding category IDs
categories = coco.loadCats(coco.getCatIds())
category_id_to_name = {cat['id']: cat['name'] for cat in categories}

# Print all category IDs and corresponding category names
for category_id, category_name in category_id_to_name.items():
    print(f"Category ID: {category_id}, Category Name: {category_name}")

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
Category ID: 0, Category Name: General trash
Category ID: 1, Category Name: Paper
Category ID: 2, Category Name: Paper pack
Category ID: 3, Category Name: Metal
Category ID: 4, Category Name: Glass
Category ID: 5, Category Name: Plastic
Category ID: 6, Category Name: Styrofoam
Category ID: 7, Category Name: Plastic bag
Category ID: 8, Category Name: Battery
Category ID: 9, Category Name: Clothing


In [27]:
from mmengine import Config
cfg = Config.fromfile('./projects/CO-DETR/configs/codino/co_dino_5scale_r50_lsj_8xb2_1x_coco_nomask.py')
# cfg = Config.fromfile('./co_dino_5scale_r50_lsj_8xb2_1x_trash.py')
print(cfg.pretty_text)

auto_scale_lr = dict(base_batch_size=16)
backend_args = None
batch_augments = [
    dict(pad_mask=False, size=(
        1024,
        1024,
    ), type='BatchFixedSizePad'),
]
custom_imports = dict(
    allow_failed_imports=False, imports=[
        'projects.CO-DETR.codetr',
    ])
data_root = 'data/coco/'
dataset_type = 'CocoDataset'
default_hooks = dict(
    checkpoint=dict(
        _scope_='mmdet',
        by_epoch=True,
        interval=1,
        max_keep_ckpts=3,
        type='CheckpointHook'),
    logger=dict(_scope_='mmdet', interval=50, type='LoggerHook'),
    param_scheduler=dict(_scope_='mmdet', type='ParamSchedulerHook'),
    sampler_seed=dict(_scope_='mmdet', type='DistSamplerSeedHook'),
    timer=dict(_scope_='mmdet', type='IterTimerHook'),
    visualization=dict(_scope_='mmdet', type='DetVisualizationHook'))
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))


In [28]:
from mmengine.runner import set_random_seed

# pre-training weight paths
cfg.load_from = './checkpoints/co_dino_5scale_r50_lsj_8xb2_1x_coco-69a72d67.pth'

# batch size
cfg.train_dataloader.batch_size = 1
# dataloader num workers
cfg.train_dataloader.num_workers = 8

# freezing the layers of the backbone network
cfg.model.backbone.frozen_stages = -1
# number of classifications
cfg.num_classes = 10

# For single card training, you need to change SyncBN to BN
#cfg.norm_cfg = dict(type='BN', requires_grad=True)

# labels and palettes(very important!!)
cfg.metainfo = {
    'classes': ("General trash", "Paper", "Paper pack", "Metal", "Glass",
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
}

# data folder
cfg.data_root = '../dataset'

# train json file path
cfg.train_dataloader.dataset.dataset.ann_file = 'json/splits/train_fold4.json'
cfg.train_dataloader.dataset.dataset.data_root = cfg.data_root
# train image file path
cfg.train_dataloader.dataset.dataset.data_prefix.img = ''
cfg.train_dataloader.dataset.dataset.metainfo = cfg.metainfo

# valid json file path
cfg.val_dataloader.dataset.ann_file = 'json/splits/val_fold4.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
# valid image file path
cfg.val_dataloader.dataset.data_prefix.img = ''
cfg.val_dataloader.dataset.metainfo = cfg.metainfo


cfg.test_dataloader = cfg.val_dataloader

# valid evaluator json file path
cfg.val_evaluator.ann_file = cfg.data_root+'/'+'json/splits/val_fold4.json'

cfg.test_evaluator = cfg.val_evaluator

# ⭐️ Set the checkpoint interval.
cfg.default_hooks.checkpoint.interval = 4

cfg.device = 'cuda'

# ⭐️ Set the evaluation interval.
#cfg.evaluation.interval = 2


# model weights are saved every 10 intervals, up to two weights are saved at the same time, and the saving strategy is auto
#cfg.default_hooks.checkpoint = dict(type='CheckpointHook', interval=10, max_keep_ckpts=2, save_best='auto')
# Interval of reporting indicators
#cfg.default_hooks.logger.interval = 1

# the ballon dataset is small, so each epoch repeats the data 4 times
#cfg.train_dataloader.dataset = dict(dict(type='RepeatDataset',times=4,dataset=cfg.train_dataloader.dataset.dataset))

# Fixed random number seed
set_random_seed(42, deterministic=False)

cfg.visualizer.vis_backends= [
    dict(type='LocalVisBackend'),
    dict(type='WandbVisBackend'),
]


#------------------------------------------------------
config=f'./custom_configs/co_dino_5scale_r50_lsj_8xb2_1x_trash.py'
with open(config, 'w') as f:
    f.write(cfg.pretty_text)

In [17]:
!python ./tools/train.py {config}

10/15 13:58:24 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 538036752
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code

In [ ]:
import os
import pandas as pd
from mmdet.apis import init_detector, inference_detector

# 모델 초기화
epoch = 20
config_file = './cascade-rcnn_x101_64x4d_fpn_20e_trash.py'  # 모델 설정 파일 경로
checkpoint_file = f'./work_dirs/cascade-rcnn_x101_64x4d_fpn_20e_trash/epoch_{epoch}.pth'  # 체크포인트 파일 경로
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# 이미지 경로 및 결과 저장 경로 설정
image_folder = '../../dataset/test'  # 이미지 폴더 경로
output_csv = './output/cascade-rcnn_output_predictions.csv'  # 출력 CSV 파일 경로

# 결과 저장 리스트
results = []

# 이미지 추론
for image_name in os.listdir(image_folder):
    if image_name.endswith(('.jpg', '.png')):  # 지원되는 이미지 형식 확인
        img_path = os.path.join(image_folder, image_name)
        result = inference_detector(model, img_path)

        # DetDataSample에서 결과 추출
        prediction_string = []
        if hasattr(result, 'pred_instances'):
            det_samples = result.pred_instances  # 예측 결과의 인스턴스들
            if det_samples is not None:
                bboxes = det_samples.bboxes
                scores = det_samples.scores
                labels = det_samples.labels

                for j in range(len(bboxes)):
                    prediction_string.append(
                        f"{int(labels[j])} {scores[j]:.4f} {bboxes[j][0]:.2f} {bboxes[j][1]:.2f} {bboxes[j][2]:.2f} {bboxes[j][3]:.2f}"
                    )

        # PredictionString을 먼저 저장하고 image_id는 그 다음에 저장
        results.append({
            'PredictionString': " ".join(prediction_string),  # 한 줄에 저장
            'image_id': f'test/{image_name}'
        })

# 데이터프레임 생성
df = pd.DataFrame(results)

# CSV 파일로 저장
df.to_csv(output_csv, index=False)
df.head()
